In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np
import datetime



import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from gensim.utils import simple_preprocess, tokenize
from gensim.summarization.textcleaner import split_sentences
from gensim.parsing import preprocessing
from gensim.models import Doc2Vec
from gensim.corpora import Dictionary


from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn import metrics # silhouette
from sklearn.preprocessing import minmax_scale

import matplotlib.pyplot as plt
from multiprocessing import cpu_count

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import ruptures as rpt
from ruptures.metrics import hausdorff, randindex

from tqdm import tqdm

# pd.set_option('display.max_colwidth',1000)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

%matplotlib inline

cuda


In [2]:
df_train = pd.read_pickle('data/prod/train_samples.pkl')

In [3]:
df_train.head()

,freqs,pivots,pattern
0,"[0.8501266384547419, 0.9333112933106216, 0.968...","[11, 63]",spike_down
1,"[0.5642528003020821, 0.5783295540701987, 0.996...",[],stable
2,"[0.9017256704805636, 0.9229583934523402, 0.986...","[16, 82]",down
3,"[0.9460213693887373, 0.8285427047594923, 0.870...","[23, 57, 92]",down_up
4,"[1.0, 0.962602833443452, 0.9951430981134859, 0...","[19, 95]",down


In [4]:
def convert_pivots(df):
    # convert pivots
    
    for i, row in df.iterrows():
        if row['pattern'] == 'up_down' or row['pattern'] == 'down_up':
            row['pivots'] = np.array([row['pivots'][0], row['pivots'][2]])
        elif row['pattern'] == 'spike_up' or row['pattern'] == 'spike_down':
            temp = []
            for p in row['pivots']:
                temp.append(p-2)
                temp.append(p+2)
            row['pivots'] = np.array(temp)
            
    return df

In [5]:
# if this has been done in data_sampling file, no need to do this again
df_train = convert_pivots(df_train)

In [6]:
X = np.stack(df_train['freqs'].values)
y = df_train['pivots'].values
z = df_train['pattern'].values

In [7]:
X_train, X_val, y_train, y_val, z_train, z_val = train_test_split(X, y, z, test_size=5000)

In [ ]:
class SeqDataset(Dataset):

    def __init__(self, X, pivots, events=None, timeline=100):
        self.X = X
        self.pivots = pivots
        self.timeline = timeline
        self.events = events

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        src = self.X[idx]
        trg = torch.zeros(self.timeline, dtype=torch.float)
        event = self.events[idx]
        
        if event == 'stable':
            is_target = torch.tensor(0, dtype=torch.float)
        else:
            is_target = torch.tensor(1, dtype=torch.float)
        
            #trg = self.format_pivots(event, self.pivots[idx])
            for i in self.pivots[idx]:
                trg[i:] = trg[i] == 0
            
        #sos = torch.tensor([1])
        #eos = torch.tensor([2])
        
        src = torch.tensor(src, dtype=torch.float)
        # src = torch.cat((sos, src, eos))
        # trg = torch.cat((sos, trg, eos))
        # add sos and eos token id
        # print(F.one_hot(trg))
        # print(self.events[idx], self.pivots[idx])
        return src, trg, is_target

        
    def format_pivots_2(self, event, pivots):
        cur = 0
        points = torch.zeros(self.timeline, dtype=torch.long)
        for p in pivots:
            if points[p] == cur:
                points[p:] = cur + 1
                cur += 1
#             else:
#                 points[p:] = 3
                
        return points
    
    def format_pivots(self, event, pivots):
        # not using anywhere but just keep here if need
        points = torch.zeros(self.timeline, dtype=torch.float)
        
        if event == 'spike_up':
            for p in pivots:
                points[p-2: p] = 1
                points[p: p+3] = 1
                
        elif event == 'spike_down':
            for p in pivots:
                points[p-2: p] = 1
                points[p: p+3] = 1
                
        elif event == 'up_down':
            points[pivots[0]: pivots[1]] = 1
            #points[pivots[1]: pivots[2]] = 1
            
        elif event == 'down_up':
            points[pivots[0]: pivots[1]] = 1
            #points[pivots[1]: pivots[2]] = 1
            
        elif event == 'up':
            points[pivots[0]: pivots[1]] = 1
            
        elif event == 'down':
            points[pivots[0]: pivots[1]] = 1
            
        else:
            pass
        
        return points


In [ ]:
train_set = SeqDataset(X_train, y_train, z_train)
val_set = SeqDataset(X_val, y_val, z_val)

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
class Seq2SeqModel(nn.Module):
    
    def __init__(self, hid_dim, out_dim, cov_dim=8, dropout=0.5, device=device):
        super(Seq2SeqModel, self).__init__()
        
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.cov_dim = cov_dim
        self.device = device
        self.embed = torch.nn.Embedding(out_dim, hid_dim)
        seq_len = 100
        
        
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0),  # (100 - 3) + 1 = 98
            nn.ReLU(True),
            nn.MaxPool1d(2, 2),  # (98 - 2) / 2 + 1 = 49
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=0), # (49 - 3) + 1 = 47
            nn.ReLU(True),
            nn.MaxPool1d(3, 2), # (47 - 3) / 2 + 1 = 23
        )
        
        # use both RNN encoded and CNN output for decoder
        
        self.seq_out = nn.Linear(hid_dim, out_dim)
        self.fc = nn.Linear(23 * 16, hid_dim)
        
        self.bin_out = nn.Linear(hid_dim, 1)
        self.dropout = nn.Dropout(dropout)
        
    
    def forward(self, src):
        batch_size = src.shape[0]
        trg_len = src.shape[1]
        
        conv_out = self.conv(src.unsqueeze(1))
        
        cnn_out = conv_out.view(batch_size, -1)
        context = self.dropout(cnn_out)
        
        fc_out = self.fc(context)
        
        seq_out = self.seq_out(F.relu(fc_out))
        bin_out = self.bin_out(F.relu(fc_out))
        
        bin_out = torch.sigmoid(bin_out)
        seq_out = torch.sigmoid(seq_out)
        
        return seq_out, bin_out


In [ ]:
#torch.save(model.state_dict(), 'models/rnn_cnn.pt')

In [ ]:
def get_result(y_bin, y_seq):
    founds = []
    # founds = np.where(y_bin > 0)[0]
    
    for i, (x, p) in enumerate(zip(y_bin, y_seq)):
        pivots = get_pivots(p)
        if len(pivots) and x:
            founds.append(i)
    return founds


def get_pivots(pred):
    # convert sequence of 0 and 1 to list of pivots
    pivots = []

    cur = pred[0]
    for i, v in enumerate(pred):
#         if v == 1 or v == 2:
#             # print(v)
#             continue
        if v != cur:
            pivots.append(i)
            cur = v
    return pivots

def visualize_train_data(X, y, y_true=[]):
    fig, ax = plt.subplots(figsize=(20, 10))
    index = np.arange(100)
    
    ax.plot(index, X)
    ax.vlines(y, ymin=0, ymax=1, color='red')
    if len(y_true):
        ax.vlines(y_true + 0.5, ymin=0, ymax=1, color='green')
    
    ax.set(xlabel='Time', ylabel='Counts')
    # ax.legend()
    ax.set_ylim(0)
    ax.grid()
    ax.set_xticks(range(0, 100, 2))

    plt.show()

    

def get_report(y_bin, y_seq, label_points, timeline=100):
    true_bin = np.array([bool(len(p)) for p in label_points])

    print(classification_report(y_bin, true_bin, digits=4))
    randidx_pos = 0
    randidx_neg = 0
    count_neg = 0
    count_pos = 0
    
    for i, true_points in enumerate(label_points):
        true_points = np.concatenate((true_points, [timeline]), axis=0).astype(int)
        
        pred_points = get_pivots(y_seq[i])
        # print(pred_points)
        if timeline not in pred_points:
            pred_points = np.concatenate((pred_points, [timeline]), axis=0).astype(int)
            
        if len(pred_points) > 1:
            count_pos += 1
            randidx_pos += randindex(true_points, pred_points)
        else:
            count_neg += 1
            randidx_neg += randindex(true_points, pred_points)
        # print(t, p)
        # print(true_points)
        
    #print(len(true_bin), true_bin.sum(), count, randidx_neg, randidx_pos)
    print("Rand Index Positive:", randidx_pos/count_pos)
    print("Rand Index Negative:", randidx_neg/count_neg)
    
    
    
    

In [ ]:
test_loss, y_bin, y_seq = test(model, val_loader)

In [ ]:
get_report(y_bin, y_seq, y_val)

In [ ]:
result = get_result(y_bin, y_seq)
print(len(result))

In [ ]:
test_id = np.random.choice(result)
print(test_id)
print(get_pivots(y_seq[test_id]))
print(y_val[test_id])

In [ ]:
visualize_train_data(X_val[test_id], get_pivots(y_seq[test_id]), y_val[test_id])